The [Wave](https://docs.python.org/3.4/library/wave.html) module handles input and output of [Wav](https://en.wikipedia.org/wiki/WAV) audio files. Audio data is represented as a bytes object, which makes Wave less useful than some other modules that return a numpy array, but it has the benefit of being part of the python standard library. 

To work with the data in the bytes object we'll also need to know the number of [bytes per sample](https://en.wikipedia.org/wiki/Audio_bit_depth) and the [sample frequency][]. These examples will use mono audio only, but the nchannels parameter tells you whether the audio is stereo or mono.

[sample frequency]: https://en.wikipedia.org/wiki/Sampling_(signal_processing)#Audio_sampling

In [ ]:
import wave

def input_wave(filename,frames=10000000): #10000000 is an arbitrary large number of frames
    with wave.open(filename,'rb') as wave_file:
        params=wave_file.getparams()
        audio=wave_file.readframes(frames)  
    return params, audio

This is a short mono audio clip for demonstration:

In [ ]:
from IPython.display import Audio
Audio(filename='wavs/Trumpet.wav')

In [ ]:
trumpet_params, trumpet_bytes = input_wave('wavs/Trumpet.wav') #must be mono
print("Parameters:", trumpet_params, "Data sample:", trumpet_bytes[:10], sep='\n')

The basic idea behind delay is to combine a sound with a repeated version. The _add_ function from another standard library module, [Audioop](https://docs.python.org/3.4/library/audioop.html), can be used to combine the original sound with a delayed copy. To add these two audio signals together, they must be the same length. So to create the delayed signal, this function will add silence to the beginning and cut off the same amount of data at the end.


So to create the audio delayed by 

In [ ]:
import audioop

def delay(audio_bytes,params,offset_ms,factor=1):
    #calculate the number of bytes which corresponds to the offset in milliseconds, 
    #depending on sampwith and framerate
    offset= params.sampwidth*offset_ms*int(params.framerate/1000)
    #create some empty  of offet-length
    beginning= b'\0'*offset
    #remove the same amount of data from the end
    end= audio_bytes[:-offset]
    return audioop.add(audio_bytes, beginning+ end, params.sampwidth)

In [ ]:
#output to file so we can use ipython notebook's Audio widget
def output_wave(audio, params, stem, suffix):
    #dynamically format the filename by passing in data
    filename=stem.replace('.wav','_{}.wav'.format(suffix))
    with wave.open(filename,'wb') as wave_file:
        wave_file.setparams(params)
        wave_file.writeframes(audio)

In [ ]:
#1-second delay
delayed_bytes=delay(trumpet_bytes,trumpet_params,1000)
output_wave(delayed_bytes, trumpet_params, 'wavs/Trumpet.wav','delay_{}'.format(1000))

In [ ]:
Audio('wavs/Trumpet_delay_1000.wav')

To make this sound more like a realistic echo, we can change the volume of the delayed audio by multiplying it with audioop.mul. Note that 

In [ ]:
#new delay function with factor
def delay(audio_bytes,params,offset_ms,factor=1):
    #calculate the number of bytes which corresponds to the offset in milliseconds, 
    #depending on sampwith and framerate
    offset= params.sampwidth*offset_ms*int(params.framerate/1000)
    #create some empty  of offet-length
    beginning= b'\0'*offset
    #remove the same amount of data from the end
    end= audio_bytes[:-offset]
    #multiply the delayed portion by a factor
    multiplied_end= audioop.mul(audio_bytes[:-offset],params.sampwidth,factor)
    return audioop.add(audio_bytes, beginning+ multiplied_end, params.sampwidth)

In [ ]:
#helper function to try out lots of delays
def delay_to_file(audio_bytes,params,offset_ms,file_stem,factor=1):
    delayed_bytes=delay(audio_bytes,params,offset_ms,factor)
    output_wave(delayed_bytes, params, file_stem,'delay_{}_{}'.format(offset_ms,factor))

A single delay of 1-2 seconds is too long to sound like a natural echo, so this can be used for a musical effect.

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=2000,file_stem='wavs/Trumpet.wav',factor=.25)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=1000,file_stem='wavs/Trumpet.wav',factor=.25)

from IPython.display import display
display(
    Audio('wavs/Trumpet_delay_2000_0.25.wav'),
    Audio('wavs/Trumpet_delay_1000_0.25.wav')
    )

Adding 250-400 ms of delay gives the impression of a natural echo that might occur in a larger physical space.

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=400, file_stem='wavs/Trumpet.wav',factor=.25)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=400, file_stem='wavs/Trumpet.wav',factor=.1)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=250, file_stem='wavs/Trumpet.wav',factor=.25)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=250, file_stem='wavs/Trumpet.wav',factor=.1)

display(
    Audio('wavs/Trumpet_delay_400_0.25.wav'),
    Audio('wavs/Trumpet_delay_400_0.1.wav'),
    Audio('wavs/Trumpet_delay_250_0.25.wav'),
    Audio('wavs/Trumpet_delay_250_0.1.wav')
    )

As delays get shorter, around 125 ms or less, they start to be percieved as a single sound rather than a distinct echo. This produces the effect of a [comb filter](https://en.wikipedia.org/wiki/Comb_filter) as certain frequencies get louder or softer due to interference. On this trumpet sample, 1-5 ms delay sounds a lot like a trumpet played through a [mute](http://www.summersong.net/teacher/trumpetlessons/brassinstrumentmutes/).

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=125,  file_stem='wavs/Trumpet.wav',factor=.25)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=75,  file_stem='wavs/Trumpet.wav',factor=.25)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=20,  file_stem='wavs/Trumpet.wav',factor=.75)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=5,  file_stem='wavs/Trumpet.wav',factor=.75)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=1,  file_stem='wavs/Trumpet.wav',factor=.75)

display(
    Audio('wavs/Trumpet_delay_125_0.25.wav'),
    Audio('wavs/Trumpet_delay_75_0.25.wav'),
    Audio('wavs/Trumpet_delay_20_0.75.wav'),
    Audio('wavs/Trumpet_delay_5_0.75.wav'),
    Audio('wavs/Trumpet_delay_1_0.75.wav')
    )       

In [ ]:
from warnings import warn

def multi_delay(audio_bytes,params,offset_ms,factor=1,num=1):
    #calculate the number of bytes which corresponds to the offset in milliseconds, 
    #depending on sampwith and framerate
    if factor>0.7 and num >3:
        warn("These settings may produce a very loud audio file. Please use caution when listening")
    offset=params.sampwidth*offset_ms*int(params.framerate/1000)
    #create a copy of the original to apply the delays
    delayed_bytes=audio_bytes
    #at each step of the loop, "
    for i in range(num):
        end = delayed_bytes[:-offset]
        #multiplied_end=end
        multiplied_end= audioop.mul(end,params.sampwidth,factor**(i+1))
        beginning = b'\0'*offset
        delayed_bytes= audioop.add(delayed_bytes, beginning+multiplied_end, params.sampwidth)
    return delayed_bytes

In [ ]:
def multi_delay_to_file(audio_bytes,params,offset_ms,file_stem,factor=1,num=1):
    echoed_bytes=multi_delay(audio_bytes,params,offset_ms,factor,num)
    output_wave(echoed_bytes, params, file_stem,'multi_delay_{}_{}_{}'.format(offset_ms,factor,num))

In [ ]:
multi_delay_to_file(trumpet_bytes,trumpet_params,offset_ms=50,file_stem='wavs/Trumpet.wav',factor=.76,num=10)
multi_delay_to_file(trumpet_bytes,trumpet_params,offset_ms=250,file_stem='wavs/Trumpet.wav',factor=.7,num=3)
multi_delay_to_file(trumpet_bytes,trumpet_params,offset_ms=3000,file_stem='wavs/Trumpet.wav',factor=.9,num=3)
display(
    Audio(filename='wavs/Trumpet_multi_delay_50_0.76_10.wav'),
    Audio(filename='wavs/Trumpet_multi_delay_250_0.7_3.wav'),
    Audio(filename='wavs/Trumpet_multi_delay_3000_0.9_3.wav'),
)

In [ ]:
# add extra space at the end for the delays
def multi_delay(audio_bytes,params,offset_ms,factor=1,num=1):
    offset=params.sampwidth*offset_ms*int(params.framerate/1000)
    longer_bytes=audio_bytes+b'\0'*offset*(num) #leave enough room for the delays
    for i in range(num):
        end = longer_bytes[:-offset*(i+1)]
        #multiplied_end=end
        multiplied_end= audioop.mul(end,params.sampwidth,factor**(i+1))
        beginning = b'\0'*offset*(i+1)
        longer_bytes= audioop.add(longer_bytes, beginning+multiplied_end, params.sampwidth)
    return longer_bytes

In [ ]:
from IPython.display import FileLinks
FileLinks('wavs/')